In [2]:
import pandas as pd
import pyBigWig
from pyfaidx import Fasta
import numpy as np
from Bio.Seq import Seq
import os
print("pandas == {}".format(pd.__version__))
print("pyBigWig == {}".format(pyBigWig.__version__))
print("pyfaidx == {}".format(__import__('pyfaidx').__version__))
print("numpy == {}".format(np.__version__))
print("biopython == {}".format(__import__('Bio').__version__))

pandas == 0.24.2
pyBigWig == 0.3.18
pyfaidx == 0.7.0
numpy == 1.16.6
biopython == 1.76


# Step 1. Extract shape signals from bw files

In [4]:
# Read annotation file
df = pd.read_csv('./genomes/hg38.gff3', sep='\t', comment='#', header=None,
                 names=['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'])

In [5]:
transcript_info_pos = df[
    (df['type'].isin(['processed_transcript', 'transcript','mRNA','miRNA','lincRNA','snRNA','RNA','snoRNA','rRNA'])) & 
    (df['strand']=="+")
]
transcript_id_pos = transcript_info_pos['attributes'].str.extract('ID=transcript:([^;]+);')[0].unique()
len(transcript_id_pos)

In [22]:
transcript_info_neg = df[
    (df['type'].isin(['processed_transcript', 'transcript','mRNA','miRNA','lincRNA','snRNA','RNA','snoRNA','rRNA'])) & 
    (df['strand']=="-")
]
transcript_id_neg = transcript_info_neg['attributes'].str.extract('ID=transcript:([^;]+);')[0].unique()
len(transcript_id_neg)

70317

In [8]:
# checking individual genes 
transcript_to_check = 'ENST00000238081'

if transcript_to_check in transcript_id_neg:
    print "{} is in transcript_info_neg.".format(transcript_to_check)
else:
    print "{} is NOT in transcript_info_neg.".format(transcript_to_check)

ENST00000238081 is in transcript_info_neg.


In [23]:
# Positive strand, invivo
genome = Fasta('./genomes/seq/hg38.fa')
# Read the file that contain the SHAPE score
bw_path = "./shape_bw_human/icSHAPE_hg38_Cell_2016_NAI-N3_invivo_plus_score_vivo_NAIN3-score-pos.bw"
bw = pyBigWig.open(bw_path)

for transcript_id in transcript_id_pos:
    # Filter exon data for the current transcript_id
    exon_data = df[(df['type'] == 'exon') & (df['attributes'].str.contains(str(transcript_id)))]
    # Extract the SHAPE score from the positive strands, without separating the exons in a list
    chrom_lengths = bw.chroms()
    # Extract the exon coordinates
    exon_coordinates = [(row.iloc[0], row.iloc[3], row.iloc[4]) for index, row in exon_data.iterrows()]
    # List to store signal arrays for each exon
    shape_sig = []
    # Iterate through each set of coordinates
    for chrom, start, end in exon_coordinates:
        if chrom in chrom_lengths and start < chrom_lengths[chrom] and end <= chrom_lengths[chrom]:
            # Retrieve the signal values for each position within the specified region
            signals = bw.values(chrom, start-1, end)  # Adjust for zero-based indexing
            # Extend the one-dimensional array with the new signals
            if signals:  # Check if signals are not None or empty
                shape_sig.extend(signals)
    if np.all(np.isnan(shape_sig)):
        continue  # Skip to the next iteration if all values in shape_sig are NaN
    # Check the number of valid signal, must be >= 10
    nan_count = np.sum(np.isnan(shape_sig))
    non_nan_count = len(shape_sig) - nan_count
    if non_nan_count < 10:
        continue
    # Check how many NAs in between the signal region
    shape_sig_array = np.array(shape_sig)
    if len(shape_sig_array) > 0:
        first_valid_index = np.where(~np.isnan(shape_sig_array))[0][0]
        last_valid_index = np.where(~np.isnan(shape_sig_array))[0][-1]
        # Calculate the total number of values and the number of NaNs in the specified range
        valid_range = shape_sig_array[first_valid_index:last_valid_index + 1]
        total_values = len(valid_range)
        num_nans = np.isnan(valid_range).sum()
        # Check if NaNs are <= 5% of the total values in the range, change it to a higher value if needed
        if num_nans > 0.05 * total_values:
            continue  # Skip to the next iteration if NaNs exceed 5% of the range
    exon_sequences = ""
    # Extract sequences for each exon
    for chrom, start, end in exon_coordinates:
        # Retrieve the sequence and concatenate it
        sequence = genome[chrom][start-1:end].seq  # pyfaidx uses 0-based start, hence start-1
        exon_sequences += sequence
    data = [(sig, 0, seq) for sig, seq in zip(shape_sig, exon_sequences)]
    # Create the DataFrame
    df2 = pd.DataFrame(data, columns=['shape_sig', 'uncert', 'exon_seq'])
    df2['shape_sig'].fillna(-999, inplace=True) # replace the nan signal with -999 (not available)
    df2 = df2.iloc[first_valid_index:last_valid_index + 1]
    df2.reset_index(drop=True, inplace=True)
    df2.index += 1
    # Create a new folder
    output_dir = './shape_extract/whole_human_pos_invivo'
    # Create the directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Define the output file path
    output_path = os.path.join(output_dir, '{}.map'.format(transcript_id))
    # Save the DataFrame to a file, named according to the current transcript_id
    df2.to_csv(output_path, sep='\t', index=True, header=False)

bw.close()
genome.close()

In [24]:
# Negative strand, invivo
genome = Fasta('./genomes/seq/hg38.fa')
bw_path = "./shape_bw_human/icSHAPE_hg38_Cell_2016_NAI-N3_invivo_minus_score_vivo_NAIN3-score-neg.bw"
bw = pyBigWig.open(bw_path)

for transcript_id in transcript_id_neg:
    # Filter exon data for the current transcript_id
    exon_data = df[(df['type'] == 'exon') & (df['attributes'].str.contains(str(transcript_id)))]
    # Extract the SHAPE score from the positive strands, without separating the exons
    chrom_lengths = bw.chroms()
    # Extract the exon coordinates
    exon_coordinates = [(row.iloc[0], row.iloc[3], row.iloc[4]) for index, row in exon_data.iterrows()]
    # List to store signal arrays for each exon
    shape_sig = []
    # Iterate through each set of coordinates
    for chrom, start, end in exon_coordinates:
        if chrom in chrom_lengths and start < chrom_lengths[chrom] and end <= chrom_lengths[chrom]:
            # Retrieve the signal values for each position within the specified region
            signals = bw.values(chrom, start-1, end)  # Adjust for zero-based indexing
            # Extend the one-dimensional array with the new signals
            if signals:  # Check if signals are not None or empty
                shape_sig.extend(signals)
    if np.all(np.isnan(shape_sig)):
        continue  # Skip to the next iteration if all values in shape_sig are NaN
    # Check the number of valid signal, must be >= 10
    nan_count = np.sum(np.isnan(shape_sig))
    non_nan_count = len(shape_sig) - nan_count
    if non_nan_count < 10:
        continue
    # Check if how many NAs are there in  the signal
    shape_sig_array = np.array(shape_sig)
    if len(shape_sig_array) > 0:
        first_valid_index = np.where(~np.isnan(shape_sig_array))[0][0]
        last_valid_index = np.where(~np.isnan(shape_sig_array))[0][-1]
        # Calculate the total number of values and the number of NaNs in the specified range
        valid_range = shape_sig_array[first_valid_index:last_valid_index + 1]
        total_values = len(valid_range)
        num_nans = np.isnan(valid_range).sum()
        # Check if NaNs are <= 5% of the total values in the range, change it to a higher value if needed
        if num_nans > 0.05 * total_values:
            continue  # Skip to the next iteration if NaNs exceed 5% of the range
    exon_sequences = ""
    # Extract sequences for each exon
    for chrom, start, end in exon_coordinates:
        # Retrieve the sequence and concatenate it
        sequence = genome[chrom][start-1:end].seq  # pyfaidx uses 0-based start, hence start-1
        exon_sequences += sequence
    # Reverse the signal
    shape_sig_R = shape_sig[::-1]
    # Reverse complement of the sequence
    seq_temp = Seq(exon_sequences)
    exon_seq_RV = str(seq_temp.reverse_complement())
    data = [(sig, 0, seq) for sig, seq in zip(shape_sig_R, exon_seq_RV)]
    # Create the DataFrame
    df2 = pd.DataFrame(data, columns=['shape_sig', 'uncert', 'exon_seq'])
    df2['shape_sig'].fillna(-999, inplace=True) # replace the nan signal with -999 (not available)
    df2 = df2.iloc[len(df2)-last_valid_index-1:len(df2)-first_valid_index]
    df2.reset_index(drop=True, inplace=True)
    df2.index += 1
    # Save the DataFrame to a file, named according to the current transcript_id
    # Create a new folder
    output_dir = './shape_extract/whole_human_neg_invivo'
    # Create the directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Define the output file path
    output_path = os.path.join(output_dir, '{}.map'.format(transcript_id))
    df2.to_csv(output_path, sep='\t', index=True, header=False)

bw.close()
genome.close()

In [25]:
# Positive strand, invitro
genome = Fasta('./genomes/seq/hg38.fa')
# Read the file that contain the SHAPE score
bw_path = "./shape_bw_human/icSHAPE_hg38_Cell_2016_NAI-N3_invitro_plus_score_vitro_NAIN3-score-pos.bw"
bw = pyBigWig.open(bw_path)

for transcript_id in transcript_id_pos:
    # Filter exon data for the current transcript_id
    exon_data = df[(df['type'] == 'exon') & (df['attributes'].str.contains(str(transcript_id)))]
    # Extract the SHAPE score from the positive strands, without separating the exons in a list
    chrom_lengths = bw.chroms()
    # Extract the exon coordinates
    exon_coordinates = [(row.iloc[0], row.iloc[3], row.iloc[4]) for index, row in exon_data.iterrows()]
    # List to store signal arrays for each exon
    shape_sig = []
    # Iterate through each set of coordinates
    for chrom, start, end in exon_coordinates:
        if chrom in chrom_lengths and start < chrom_lengths[chrom] and end <= chrom_lengths[chrom]:
            # Retrieve the signal values for each position within the specified region
            signals = bw.values(chrom, start-1, end)  # Adjust for zero-based indexing
            # Extend the one-dimensional array with the new signals
            if signals:  # Check if signals are not None or empty
                shape_sig.extend(signals)
    if np.all(np.isnan(shape_sig)):
        continue  # Skip to the next iteration if all values in shape_sig are NaN
    nan_count = np.sum(np.isnan(shape_sig))
    non_nan_count = len(shape_sig) - nan_count
    if non_nan_count < 10:
        continue
    # Check how many NAs in between the signal region
    shape_sig_array = np.array(shape_sig)
    if len(shape_sig_array) > 0:
        first_valid_index = np.where(~np.isnan(shape_sig_array))[0][0]
        last_valid_index = np.where(~np.isnan(shape_sig_array))[0][-1]
        # Calculate the total number of values and the number of NaNs in the specified range
        valid_range = shape_sig_array[first_valid_index:last_valid_index + 1]
        total_values = len(valid_range)
        num_nans = np.isnan(valid_range).sum()
        # Check if NaNs are <= 5% of the total values in the range, change it to a higher value if needed
        if num_nans > 0.05 * total_values:
            continue  # Skip to the next iteration if NaNs exceed 5% of the range
    exon_sequences = ""
    # Extract sequences for each exon
    for chrom, start, end in exon_coordinates:
        # Retrieve the sequence and concatenate it
        sequence = genome[chrom][start-1:end].seq  # pyfaidx uses 0-based start, hence start-1
        exon_sequences += sequence
    data = [(sig, 0, seq) for sig, seq in zip(shape_sig, exon_sequences)]
    # Create the DataFrame
    df2 = pd.DataFrame(data, columns=['shape_sig', 'uncert', 'exon_seq'])
    df2['shape_sig'].fillna(-999, inplace=True) # replace the nan signal with -999 (not available)
    df2 = df2.iloc[first_valid_index:last_valid_index + 1]
    df2.reset_index(drop=True, inplace=True)
    df2.index += 1
    # Create a new folder
    output_dir = './shape_extract/whole_human_pos_invitro'
    # Create the directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Define the output file path
    output_path = os.path.join(output_dir, '{}.map'.format(transcript_id))
    # Save the DataFrame to a file, named according to the current transcript_id
    df2.to_csv(output_path, sep='\t', index=True, header=False)

bw.close()
genome.close()

In [26]:
# Negative strand, invitro
genome = Fasta('./genomes/seq/hg38.fa')
bw_path = "./shape_bw_human/icSHAPE_hg38_Cell_2016_NAI-N3_invitro_minus_score_vitro_NAIN3-score-neg.bw"
bw = pyBigWig.open(bw_path)

for transcript_id in transcript_id_neg:
    # Filter exon data for the current transcript_id
    exon_data = df[(df['type'] == 'exon') & (df['attributes'].str.contains(str(transcript_id)))]
    # Extract the SHAPE score from the positive strands, without separating the exons
    chrom_lengths = bw.chroms()
    # Extract the exon coordinates
    exon_coordinates = [(row.iloc[0], row.iloc[3], row.iloc[4]) for index, row in exon_data.iterrows()]
    # List to store signal arrays for each exon
    shape_sig = []
    # Iterate through each set of coordinates
    for chrom, start, end in exon_coordinates:
        if chrom in chrom_lengths and start < chrom_lengths[chrom] and end <= chrom_lengths[chrom]:
            # Retrieve the signal values for each position within the specified region
            signals = bw.values(chrom, start-1, end)  # Adjust for zero-based indexing
            # Extend the one-dimensional array with the new signals
            if signals:  # Check if signals are not None or empty
                shape_sig.extend(signals)
    if np.all(np.isnan(shape_sig)):
        continue  # Skip to the next iteration if all values in shape_sig are NaN
    # Check the number of valid signal, must be >= 10
    nan_count = np.sum(np.isnan(shape_sig))
    non_nan_count = len(shape_sig) - nan_count
    if non_nan_count < 10:
        continue
    # Check if how many NAs are there in  the signal
    shape_sig_array = np.array(shape_sig)
    if len(shape_sig_array) > 0:
        first_valid_index = np.where(~np.isnan(shape_sig_array))[0][0]
        last_valid_index = np.where(~np.isnan(shape_sig_array))[0][-1]
        # Calculate the total number of values and the number of NaNs in the specified range
        valid_range = shape_sig_array[first_valid_index:last_valid_index + 1]
        total_values = len(valid_range)
        num_nans = np.isnan(valid_range).sum()
        # Check if NaNs are <= 5% of the total values in the range, change it to a higher value if needed
        if num_nans > 0.05 * total_values:
            continue  # Skip to the next iteration if NaNs exceed 5% of the range
    exon_sequences = ""
    # Extract sequences for each exon
    for chrom, start, end in exon_coordinates:
        # Retrieve the sequence and concatenate it
        sequence = genome[chrom][start-1:end].seq  # pyfaidx uses 0-based start, hence start-1
        exon_sequences += sequence
    # Reverse the signal
    shape_sig_R = shape_sig[::-1]
    # Reverse complement of the sequence
    seq_temp = Seq(exon_sequences)
    exon_seq_RV = str(seq_temp.reverse_complement())
    data = [(sig, 0, seq) for sig, seq in zip(shape_sig_R, exon_seq_RV)]
    # Create the DataFrame
    df2 = pd.DataFrame(data, columns=['shape_sig', 'uncert', 'exon_seq'])
    df2['shape_sig'].fillna(-999, inplace=True) # replace the nan signal with -999 (not available)
    df2 = df2.iloc[len(df2)-last_valid_index-1:len(df2)-first_valid_index]
    df2.reset_index(drop=True, inplace=True)
    df2.index += 1
    # Save the DataFrame to a file, named according to the current transcript_id
    # Create a new folder
    output_dir = './shape_extract/whole_human_neg_invitro'
    # Create the directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Define the output file path
    output_path = os.path.join(output_dir, '{}.map'.format(transcript_id))
    df2.to_csv(output_path, sep='\t', index=True, header=False)

bw.close()
genome.close()